# Making a mock observation from the EAGLE simulations

This script is an example script to lay out the individual steps of making mock observations.

The steps are as follows:

1. Load the EAGLE simulated data.  Specify the distance at which we want to observe the cosmic web i.e. redshift of central wavelength of filter and the resolution that the data should be binned to.  The minimum resolution at 50Mpc away is about 14".
2. Cut out a chunk of the simulated data that corresponds to the Dragonfly FOV.  (switched with step 3 because takes too long for the whole simulation)
3. Add the noise to the simulated data to mimic observational noises (includes sky background noise, read out noise, dark current noise, and shot noise).  This depends on the exposure time of the mock observation.
4. Plot the data to check and see what it looks like.

Ta da!

In [1]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import os
import get_halpha_SB

%matplotlib inline

%run 'load_data.ipynb'
%run 'add_noise.ipynb'
%run 'extract_FOV.ipynb'

### Step 1:  Load the data (load_data.ipynb)
Let's say we want to observe at 100 Mpc away, with a resolution of 100 arcsec.  Our filter width is 3nm which corresponds to about 20 Mpc in radial distance (so we specify a slice width of 20).

In [2]:
distance = '50Mpc'; resolution = 14.; slicewidth = 20; machine='coho'
data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance,slicewidth=slicewidth)

Loading data of slicewidth 20 (after rounding), number of slices is 4
data_50Mpc_14arcsec_20slwd.npz exists (and is special because cant save as tuple), loading now...


In [3]:
#total_fname = 'data_50Mpc_14arcsec_20slwd.npz'  # copied over from emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_20slice_zcen12.5_total.npz
#sl = [slice(None,None,None), slice(None,None,None)]
#data = (np.load(total_fname)['arr_0'])[sl]
#data_tuple = (data,32000,1)
##np.savez(total_fname,data) ### Seems to by a python issue that cannot save large numpy object arrays, cPickle breaks or something

### Step 2: Extract the Dragonfly FOV (extract_FOV.ipynb)

In [4]:
#data_FOV_50Mpc, xystarts_50, size_50 = extractFOV(data_tuple,resolution,distance)

#distance = '46Mpc'  # ('92Mpc',) actual distance that corresponds to 14" per pixel in the simulation 
#data_FOV_46Mpc, xystarts_46, size_46 = extractFOV(data_tuple,resolution,distance)

#data_FOV_50Mpc.shape
#data_FOV_46Mpc.shape

#fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12, 12))
#get_halpha_SB.makemap(data_FOV_50Mpc,size_50,ax1,xystarts = xystarts_50)
#get_halpha_SB.makemap(data_FOV_46Mpc,size_46,ax2,xystarts = xystarts_46)

### Step 3: Add noise to the data (add_noise.ipynb)
Let's say that we want to observe for 1000 hours.

In [ ]:
exptime = 1000.*60.*60.
debugging = True; R_squared = None; CMOS = False
noiseadded_signal_50,B_sky_array_50,R_array_50 = addnoise(data_FOV_50Mpc,resolution,exptime=exptime,debugging=True)#,expnum=1)
# default for addnoise: oldcam == old QE and old R

### Step 5: Plot the data


In [ ]:
def plthist(ax3,B_sky,R_squared,R,title='',label1='',label2='',label3=''):
    ax3.hist(np.ravel(B_sky),edgecolor='black', alpha = 0.5,label=label1)
    ax3.hist(np.ravel(R_squared),edgecolor='black', alpha = 0.5,label=label2)
    #ax3.hist(np.ravel(R),edgecolor='black', alpha = 0.5, label=label3)
    ax3.text(0.6,0.7,r'$\sigma$_sb = %.1f'%(np.std(B_sky)),transform=ax3.transAxes)
    ax3.text(0.6,0.6,r'$\sigma$_r = %.1f'%(np.std(R_squared)),transform=ax3.transAxes)
    ax3.text(0.6,0.5,r'$\sigma$_r = %.1f'%(np.std(R)),transform=ax3.transAxes)
    #ax3.set_ylim(0,8500)
    #ax3.set_xlim(-1500,2000)
    ax3.set_title(title)

fig, (ax1) = plt.subplots(1, 1, figsize=(6, 6))
plthist(ax1,B_sky_array,R_array**2,R_array,label1='b_sky',label2='r_squared',label3='r')
ax1.legend()
print B_sky_array.shape
print R_array

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(6, 6))
get_halpha_SB.makemap(noiseadded_signal,size,ax1,xystarts = xystarts)